# LSTM -- SOLUTION

In [16]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

In [17]:
#device= torch.device("cuda")
device= torch.device("cpu")
print(device)

cpu


### Download Penn Tree Bank (the tensor train_data should consists of 20 columns of ~50,000 words)

In [18]:
train_data  =  torch.load('../../Data/tao_tensors/train_data.pt')
test_data   =  torch.load('../../Data/tao_tensors/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([124656, 10])
torch.Size([53424, 10])


### Some constants associated with the data set

In [19]:
bs = 20
vocab_size = 10000


### Make a recurrent net class

In [20]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.LSTM(      hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      =   self.layer1( word_seq )  
        h_seq , (h_final,c_final)  =   self.layer2( g_seq , (h_init,c_init) )      
        score_seq                  =   self.layer3( h_seq )
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [21]:
hidden_size=300
net = three_layer_recurrent_net( hidden_size )
print(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): LSTM(300, 300)
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)


### Send the weights of the networks to the GPU

In [22]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [23]:
net.layer1.weight.data.uniform_(-0.1, 0.1)
net.layer3.weight.data.uniform_(-0.1, 0.1)
print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [24]:
criterion = nn.CrossEntropyLoss()
my_lr = 5
seq_length = 35

### Function to evaluate the network on the test set

In [25]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
    
    with torch.no_grad():
       
        h = torch.zeros(1, bs, hidden_size)
        c = torch.zeros(1, bs, hidden_size)

        h=h.to(device)
        c=c.to(device)

        for count in range( 0 , 4120-seq_length ,  seq_length) :

            minibatch_data =  test_data[ count   : count+seq_length   ]
            minibatch_label = test_data[ count+1 : count+seq_length+1 ]

            minibatch_data=minibatch_data.to(device)
            minibatch_label=minibatch_label.to(device)

            scores, h, c  = net( minibatch_data, h , c)

            minibatch_label =   minibatch_label.view(  bs*seq_length ) 
            scores          =            scores.view(  bs*seq_length , vocab_size)

            loss = criterion(  scores ,  minibatch_label )    

            h=h.detach()
            c=c.detach()

            running_loss += loss.item()
            num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [26]:
start=time.time()

for epoch in range(8):
    
    # divide the learning rate by 3 except after the first epoch
    if epoch >= 2:
        my_lr = my_lr / 3
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        c=c.detach()
        h=h.requires_grad_()
        c=c.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h, c  = net( minibatch_data, h , c)
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got CPUDoubleTensor instead (while checking arguments for embedding)

### Choose one sentence (taken from the test set)

In [12]:
# SENTENCES FROM TEST SET

sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence2 = "some analysts expect oil prices to remain relatively"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# OR MAKE YOUR OWN SENTENCE. 
# NO CAPITAL LETTER ALLOWED. EACH WORD MUST BE IN THE ALLOWED VOCABULARY OF 10,000 WORDS

sentence6= "he was very"


# CHOOSE THE SENTENCE
mysentence = sentence1

### Convert the sentence into a vector, then send to GPU

In [13]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[  307],
        [ 1140],
        [  334],
        [ 1486],
        [ 1786],
        [   64],
        [  719],
        [  377]], device='cuda:0')


### Set the initial hidden state to zero, then run the LSTM.

In [14]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [15]:
print(mysentence, '... \n')

utils.show_next_word(scores)

some analysts expect oil prices to remain relatively ... 

9.8%	 high
7.2%	 low
4.2%	 flat
4.2%	 <unk>
3.7%	 thin
2.9%	 strong
2.5%	 well
2.3%	 heavy
1.8%	 modest
1.4%	 good
1.2%	 much
1.1%	 bullish
1.1%	 moderate
1.1%	 profitable
1.0%	 greater
0.9%	 full
0.9%	 little
0.8%	 adjusted
0.8%	 higher
0.7%	 positive
0.7%	 narrow
0.6%	 lower
0.6%	 slow
0.6%	 less
0.5%	 robust
0.5%	 active
0.5%	 attractive
0.5%	 traded
0.5%	 tight
0.4%	 small
